In [1]:
import os
import json
import pandas as pd
import pymysql
import plotly.express as px
import plotly.graph_objects as go

In [2]:
if "df_agg_transaction" in globals():
    del df_agg_transaction
if "agg_txn_records" in globals():
    del agg_txn_records

# -----------------------------
# 1. SET BASE PATH
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_TRANSACTION = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/aggregated/transaction/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
agg_txn_records = []

for state_folder in os.listdir(BASE_PATH_TRANSACTION):
    state_path = os.path.join(BASE_PATH_TRANSACTION, state_folder)
    if not os.path.isdir(state_path):
        continue  # skip files

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        # year like "2018", "2019"
        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]  # "1" from "1.json"
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            transaction_block = data.get("data", {}).get("transactionData", None)
            if not transaction_block:
                continue

            for entry in transaction_block:
                txn_type = entry.get("name")
                instruments = entry.get("paymentInstruments", [])
                if not instruments:
                    continue

                count = instruments[0].get("count")
                amount = instruments[0].get("amount")

                agg_txn_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "txn_type": txn_type,
                    "txn_count": count,
                    "txn_amount": amount
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_agg_transaction = pd.DataFrame(agg_txn_records)

# -----------------------------
# 4. CLEAN STATE NAMES
# -----------------------------
df_agg_transaction["state"] = (
    df_agg_transaction["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_agg_transaction["state"] = df_agg_transaction["state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_agg_transaction["year"] = df_agg_transaction["year"].astype(int)
df_agg_transaction["quarter"] = df_agg_transaction["quarter"].astype(int)
df_agg_transaction["txn_count"] = df_agg_transaction["txn_count"].astype("int64")
df_agg_transaction["txn_amount"] = df_agg_transaction["txn_amount"].astype("float64")

# OPTIONAL: drop exact duplicates if any
df_agg_transaction = df_agg_transaction.drop_duplicates()

# SORT & RESET INDEX
df_agg_transaction = df_agg_transaction.sort_values(
    by=["state", "year", "quarter", "txn_type"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_agg_transaction.head())
print("Shape:", df_agg_transaction.shape)
df_agg_transaction.info()
print("Duplicate rows:", df_agg_transaction.duplicated().sum())

,state_raw,year,quarter,txn_type,txn_count,txn_amount,state
0,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04,Andaman & Nicobar
1,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05,Andaman & Nicobar
2,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05,Andaman & Nicobar
3,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07,Andaman & Nicobar
4,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06,Andaman & Nicobar


Shape: (5034, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5034 entries, 0 to 5033
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_raw   5034 non-null   object 
 1   year        5034 non-null   int64  
 2   quarter     5034 non-null   int64  
 3   txn_type    5034 non-null   object 
 4   txn_count   5034 non-null   int64  
 5   txn_amount  5034 non-null   float64
 6   state       5034 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 275.4+ KB
Duplicate rows: 0


In [3]:
# -----------------------------
# 0. CLEAR OLD VARIABLES (SAFE RE-RUN)
# -----------------------------
if "df_agg_user" in globals():
    del df_agg_user
if "agg_user_records" in globals():
    del agg_user_records

# -----------------------------
# 1. SET BASE PATH FOR AGGREGATED USER
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_USER = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/aggregated/user/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
agg_user_records = []

for state_folder in os.listdir(BASE_PATH_USER):
    state_path = os.path.join(BASE_PATH_USER, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        # year like "2018", "2019", etc.
        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]  # "1" from "1.json"
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            data_block = data.get("data", {})

            # Total registered users and app opens (if present)
            agg_info = data_block.get("aggregated", {})
            total_registered_users = agg_info.get("registeredUsers")
            total_app_opens = agg_info.get("appOpens")

            # Device-wise details
            users_by_device = data_block.get("usersByDevice", [])

            # Some early quarters may not have usersByDevice
            if not users_by_device:
                continue  # skip if no breakdown

            for entry in users_by_device:
                brand = entry.get("brand")
                count = entry.get("count")
                percentage = entry.get("percentage")

                agg_user_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "device_brand": brand,
                    "device_user_count": count,
                    "device_percentage": percentage,
                    "total_registered_users": total_registered_users,
                    "total_app_opens": total_app_opens
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_agg_user = pd.DataFrame(agg_user_records)

# -----------------------------
# 4. CLEAN STATE NAMES (SAME LOGIC AS TRANSACTION)
# -----------------------------
df_agg_user["state"] = (
    df_agg_user["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_agg_user["state"] = df_agg_user["state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_agg_user["year"] = df_agg_user["year"].astype(int)
df_agg_user["quarter"] = df_agg_user["quarter"].astype(int)
df_agg_user["device_user_count"] = df_agg_user["device_user_count"].astype("int64")

# Some totals might be None in some files; handle them safely
df_agg_user["total_registered_users"] = pd.to_numeric(
    df_agg_user["total_registered_users"], errors="coerce"
).astype("Int64")  # nullable integer

df_agg_user["total_app_opens"] = pd.to_numeric(
    df_agg_user["total_app_opens"], errors="coerce"
).astype("Int64")

df_agg_user["device_percentage"] = pd.to_numeric(
    df_agg_user["device_percentage"], errors="coerce"
).astype("float64")

# OPTIONAL: drop duplicates
df_agg_user = df_agg_user.drop_duplicates()

# SORT & RESET INDEX
df_agg_user = df_agg_user.sort_values(
    by=["state", "year", "quarter", "device_brand"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_agg_user.head())
print("Shape:", df_agg_user.shape)
df_agg_user.info()
print("Duplicate rows:", df_agg_user.duplicated().sum())


,state_raw,year,quarter,device_brand,device_user_count,device_percentage,total_registered_users,total_app_opens,state
0,andaman-&-nicobar-islands,2018,1,Apple,229,0.033976,6740,0,Andaman & Nicobar
1,andaman-&-nicobar-islands,2018,1,Huawei,158,0.023442,6740,0,Andaman & Nicobar
2,andaman-&-nicobar-islands,2018,1,Lenovo,202,0.029970,6740,0,Andaman & Nicobar
3,andaman-&-nicobar-islands,2018,1,Motorola,226,0.033531,6740,0,Andaman & Nicobar
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258,6740,0,Andaman & Nicobar


Shape: (6732, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state_raw               6732 non-null   object 
 1   year                    6732 non-null   int64  
 2   quarter                 6732 non-null   int64  
 3   device_brand            6732 non-null   object 
 4   device_user_count       6732 non-null   int64  
 5   device_percentage       6732 non-null   float64
 6   total_registered_users  6732 non-null   Int64  
 7   total_app_opens         6732 non-null   Int64  
 8   state                   6732 non-null   object 
dtypes: Int64(2), float64(1), int64(3), object(3)
memory usage: 486.6+ KB
Duplicate rows: 0


In [4]:
# -----------------------------
# 0. CLEAR OLD VARIABLES (SAFE RE-RUN)
# -----------------------------
if "df_agg_insurance" in globals():
    del df_agg_insurance
if "agg_ins_records" in globals():
    del agg_ins_records

# -----------------------------
# 1. SET BASE PATH FOR AGGREGATED INSURANCE
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_INS = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/aggregated/insurance/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
agg_ins_records = []

for state_folder in os.listdir(BASE_PATH_INS):
    state_path = os.path.join(BASE_PATH_INS, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        # year like "2018", "2019" ...
        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            transaction_block = data.get("data", {}).get("transactionData", None)
            if not transaction_block:
                continue

            for entry in transaction_block:
                ins_type = entry.get("name")
                instruments = entry.get("paymentInstruments", [])
                if not instruments:
                    continue

                count = instruments[0].get("count")
                amount = instruments[0].get("amount")

                agg_ins_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "ins_type": ins_type,
                    "ins_count": count,
                    "ins_amount": amount
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_agg_insurance = pd.DataFrame(agg_ins_records)

# -----------------------------
# 4. CLEAN STATE NAMES (SAME RULES)
# -----------------------------
df_agg_insurance["state"] = (
    df_agg_insurance["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_agg_insurance["state"] = df_agg_insurance["state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_agg_insurance["year"] = df_agg_insurance["year"].astype(int)
df_agg_insurance["quarter"] = df_agg_insurance["quarter"].astype(int)
df_agg_insurance["ins_count"] = df_agg_insurance["ins_count"].astype("int64")
df_agg_insurance["ins_amount"] = df_agg_insurance["ins_amount"].astype("float64")

# OPTIONAL: drop duplicates
df_agg_insurance = df_agg_insurance.drop_duplicates()

# SORT & RESET INDEX
df_agg_insurance = df_agg_insurance.sort_values(
    by=["state", "year", "quarter", "ins_type"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_agg_insurance.head())
print("Shape:", df_agg_insurance.shape)
df_agg_insurance.info()
print("Duplicate rows:", df_agg_insurance.duplicated().sum())


,state_raw,year,quarter,ins_type,ins_count,ins_amount,state
0,andaman-&-nicobar-islands,2020,2,Insurance,6,1360.0,Andaman & Nicobar
1,andaman-&-nicobar-islands,2020,3,Insurance,41,15380.0,Andaman & Nicobar
2,andaman-&-nicobar-islands,2020,4,Insurance,124,157975.0,Andaman & Nicobar
3,andaman-&-nicobar-islands,2021,1,Insurance,225,244266.0,Andaman & Nicobar
4,andaman-&-nicobar-islands,2021,2,Insurance,137,181504.0,Andaman & Nicobar


Shape: (682, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_raw   682 non-null    object 
 1   year        682 non-null    int64  
 2   quarter     682 non-null    int64  
 3   ins_type    682 non-null    object 
 4   ins_count   682 non-null    int64  
 5   ins_amount  682 non-null    float64
 6   state       682 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 37.4+ KB
Duplicate rows: 0


In [5]:
# -----------------------------
# 0. CLEAR OLD VARIABLES (SAFE RE-RUN)
# -----------------------------
if "df_map_transaction" in globals():
    del df_map_transaction
if "map_txn_records" in globals():
    del map_txn_records

# -----------------------------
# 1. SET BASE PATH FOR MAP TRANSACTION
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_MAP_TXN = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/map/transaction/hover/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
map_txn_records = []

for state_folder in os.listdir(BASE_PATH_MAP_TXN):
    state_path = os.path.join(BASE_PATH_MAP_TXN, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # For map/transaction, structure is usually: data["data"]["hoverDataList"]
            hover_list = data.get("data", {}).get("hoverDataList", None)
            if not hover_list:
                continue

            for entry in hover_list:
                district_name = entry.get("name")
                metrics = entry.get("metric", [])
                if not metrics:
                    continue

                count = metrics[0].get("count")
                amount = metrics[0].get("amount")

                map_txn_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "district": district_name,
                    "district_txn_count": count,
                    "district_txn_amount": amount
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_map_transaction = pd.DataFrame(map_txn_records)

# -----------------------------
# 4. CLEAN STATE NAMES (SAME LOGIC)
# -----------------------------
df_map_transaction["state"] = (
    df_map_transaction["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_map_transaction["state"] = df_map_transaction["state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_map_transaction["year"] = df_map_transaction["year"].astype(int)
df_map_transaction["quarter"] = df_map_transaction["quarter"].astype(int)
df_map_transaction["district_txn_count"] = df_map_transaction["district_txn_count"].astype("int64")
df_map_transaction["district_txn_amount"] = df_map_transaction["district_txn_amount"].astype("float64")

# OPTIONAL: drop duplicates
df_map_transaction = df_map_transaction.drop_duplicates()

# SORT & RESET INDEX
df_map_transaction = df_map_transaction.sort_values(
    by=["state", "year", "quarter", "district"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_map_transaction.head())
print("Shape:", df_map_transaction.shape)
df_map_transaction.info()
print("Duplicate rows:", df_map_transaction.duplicated().sum())


,state_raw,year,quarter,district,district_txn_count,district_txn_amount,state
0,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06,Andaman & Nicobar
1,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05,Andaman & Nicobar
2,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07,Andaman & Nicobar
3,andaman-&-nicobar-islands,2018,2,nicobars district,1120,3.072437e+06,Andaman & Nicobar
4,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06,Andaman & Nicobar


Shape: (20604, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20604 entries, 0 to 20603
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   state_raw            20604 non-null  object 
 1   year                 20604 non-null  int64  
 2   quarter              20604 non-null  int64  
 3   district             20604 non-null  object 
 4   district_txn_count   20604 non-null  int64  
 5   district_txn_amount  20604 non-null  float64
 6   state                20604 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 1.1+ MB
Duplicate rows: 0


In [6]:
# -----------------------------
# 0. CLEAR OLD VARIABLES (SAFE RE-RUN)
# -----------------------------
if "df_map_user" in globals():
    del df_map_user
if "map_user_records" in globals():
    del map_user_records

# -----------------------------
# 1. SET BASE PATH FOR MAP USER
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_MAP_USER = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/map/user/hover/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
map_user_records = []

for state_folder in os.listdir(BASE_PATH_MAP_USER):
    state_path = os.path.join(BASE_PATH_MAP_USER, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            hover_data = data.get("data", {}).get("hoverData", None)
            if not hover_data:
                continue

            # hoverData is a dict: { "DistrictName": {registeredUsers: ..., appOpens: ...}, ... }
            for district_name, metrics in hover_data.items():
                registered_users = metrics.get("registeredUsers")
                app_opens = metrics.get("appOpens")

                map_user_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "district": district_name,
                    "registered_users": registered_users,
                    "app_opens": app_opens
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_map_user = pd.DataFrame(map_user_records)

# -----------------------------
# 4. CLEAN STATE NAMES (SAME LOGIC)
# -----------------------------
df_map_user["state"] = (
    df_map_user["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_map_user["state"] = df_map_user["state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_map_user["year"] = df_map_user["year"].astype(int)
df_map_user["quarter"] = df_map_user["quarter"].astype(int)
df_map_user["registered_users"] = df_map_user["registered_users"].astype("int64")
df_map_user["app_opens"] = df_map_user["app_opens"].astype("int64")

# OPTIONAL: drop duplicates
df_map_user = df_map_user.drop_duplicates()

# SORT & RESET INDEX
df_map_user = df_map_user.sort_values(
    by=["state", "year", "quarter", "district"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_map_user.head())
print("Shape:", df_map_user.shape)
df_map_user.info()
print("Duplicate rows:", df_map_user.duplicated().sum())


,state_raw,year,quarter,district,registered_users,app_opens,state
0,andaman-&-nicobar-islands,2018,1,nicobars district,262,0,Andaman & Nicobar
1,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0,Andaman & Nicobar
2,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0,Andaman & Nicobar
3,andaman-&-nicobar-islands,2018,2,nicobars district,351,0,Andaman & Nicobar
4,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0,Andaman & Nicobar


Shape: (20608, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20608 entries, 0 to 20607
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   state_raw         20608 non-null  object
 1   year              20608 non-null  int64 
 2   quarter           20608 non-null  int64 
 3   district          20608 non-null  object
 4   registered_users  20608 non-null  int64 
 5   app_opens         20608 non-null  int64 
 6   state             20608 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.1+ MB
Duplicate rows: 0


In [7]:
# -----------------------------
# 0. CLEAR OLD VARIABLES
# -----------------------------
if "df_top_transaction" in globals():
    del df_top_transaction
if "top_txn_records" in globals():
    del top_txn_records

# -----------------------------
# 1. SET BASE PATH FOR TOP TRANSACTION
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_TOP_TXN = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/top/transaction/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
top_txn_records = []

for state_folder in os.listdir(BASE_PATH_TOP_TXN):
    state_path = os.path.join(BASE_PATH_TOP_TXN, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            data_block = data.get("data", {})

            # Handle pincodes
            for entry in data_block.get("pincodes", []):
                name = entry.get("entityName")
                metric = entry.get("metric", {})
                count = metric.get("count")
                amount = metric.get("amount")

                top_txn_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "entity_type": "pincode",
                    "entity_name": name,
                    "txn_count": count,
                    "txn_amount": amount
                })

            # Optionally handle districts if available
            for entry in data_block.get("districts", []):
                name = entry.get("entityName")
                metric = entry.get("metric", {})
                count = metric.get("count")
                amount = metric.get("amount")

                top_txn_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "entity_type": "district",
                    "entity_name": name,
                    "txn_count": count,
                    "txn_amount": amount
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_top_transaction = pd.DataFrame(top_txn_records)

# -----------------------------
# 4. CLEAN STATE NAMES
# -----------------------------
df_top_transaction["parent_state"] = (
    df_top_transaction["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_top_transaction["parent_state"] = df_top_transaction["parent_state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_top_transaction["year"] = df_top_transaction["year"].astype(int)
df_top_transaction["quarter"] = df_top_transaction["quarter"].astype(int)
df_top_transaction["txn_count"] = df_top_transaction["txn_count"].astype("int64")
df_top_transaction["txn_amount"] = df_top_transaction["txn_amount"].astype("float64")

# OPTIONAL: drop duplicates
df_top_transaction = df_top_transaction.drop_duplicates()

# SORT & RESET INDEX
df_top_transaction = df_top_transaction.sort_values(
    by=["parent_state", "year", "quarter", "entity_type", "entity_name"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_top_transaction.head())
print("Shape:", df_top_transaction.shape)
df_top_transaction.info()
print("Duplicate rows:", df_top_transaction.duplicated().sum())


,state_raw,year,quarter,entity_type,entity_name,txn_count,txn_amount,parent_state
0,andaman-&-nicobar-islands,2018,1,district,nicobars,528,1.139849e+06,Andaman & Nicobar
1,andaman-&-nicobar-islands,2018,1,district,north and middle andaman,442,9.316631e+05,Andaman & Nicobar
2,andaman-&-nicobar-islands,2018,1,district,south andaman,5688,1.256025e+07,Andaman & Nicobar
3,andaman-&-nicobar-islands,2018,1,pincode,744101,1622,2.769298e+06,Andaman & Nicobar
4,andaman-&-nicobar-islands,2018,1,pincode,744102,969,3.519060e+06,Andaman & Nicobar


Shape: (18295, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18295 entries, 0 to 18294
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state_raw     18295 non-null  object 
 1   year          18295 non-null  int64  
 2   quarter       18295 non-null  int64  
 3   entity_type   18295 non-null  object 
 4   entity_name   18293 non-null  object 
 5   txn_count     18295 non-null  int64  
 6   txn_amount    18295 non-null  float64
 7   parent_state  18295 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 1.1+ MB
Duplicate rows: 0


In [8]:
# -----------------------------
# 0. CLEAR OLD VARIABLES
# -----------------------------
if "df_top_user" in globals():
    del df_top_user
if "top_user_records" in globals():
    del top_user_records

# -----------------------------
# 1. SET BASE PATH FOR TOP USER
# -----------------------------
# 👉 CHANGE THIS TO YOUR ACTUAL PATH
BASE_PATH_TOP_USER = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/top/user/country/india/state"

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
top_user_records = []

for state_folder in os.listdir(BASE_PATH_TOP_USER):
    state_path = os.path.join(BASE_PATH_TOP_USER, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            # Read JSON
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            data_block = data.get("data", {})

            # Handle pincodes
            for entry in data_block.get("pincodes", []):
                # Some schemas use "name", some "entityName" – handle both
                name = entry.get("name") or entry.get("entityName")
                registered = entry.get("registeredUsers")

                top_user_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "entity_type": "pincode",
                    "entity_name": name,
                    "registered_users": registered
                })

            # Optionally handle states if present
            states_list = data_block.get("states") or []   # if None → []
            for entry in states_list:
                name = entry.get("name") or entry.get("entityName")
                registered = entry.get("registeredUsers")

                top_user_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "entity_type": "state",
                    "entity_name": name,
                    "registered_users": registered
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_top_user = pd.DataFrame(top_user_records)

# -----------------------------
# 4. CLEAN STATE NAMES
# -----------------------------
df_top_user["parent_state"] = (
    df_top_user["state_raw"]
      .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
      .str.replace("-", " ", regex=False)
      .str.title()
)

df_top_user["parent_state"] = df_top_user["parent_state"].str.replace(
    "Dadra & Nagar Haveli & Daman & Diu",
    "Dadra And Nagar Haveli And Daman And Diu",
    regex=False
)

# -----------------------------
# 5. FIX DATA TYPES
# -----------------------------
df_top_user["year"] = df_top_user["year"].astype(int)
df_top_user["quarter"] = df_top_user["quarter"].astype(int)
df_top_user["registered_users"] = df_top_user["registered_users"].astype("int64")

# OPTIONAL: drop duplicates
df_top_user = df_top_user.drop_duplicates()

# SORT & RESET INDEX
df_top_user = df_top_user.sort_values(
    by=["parent_state", "year", "quarter", "entity_type", "entity_name"]
).reset_index(drop=True)

# -----------------------------
# 6. QUICK CHECKS
# -----------------------------
display(df_top_user.head())
print("Shape:", df_top_user.shape)
df_top_user.info()
print("Duplicate rows:", df_top_user.duplicated().sum())


,state_raw,year,quarter,entity_type,entity_name,registered_users,parent_state
0,andaman-&-nicobar-islands,2018,1,pincode,744101,1108,Andaman & Nicobar
1,andaman-&-nicobar-islands,2018,1,pincode,744102,1006,Andaman & Nicobar
2,andaman-&-nicobar-islands,2018,1,pincode,744103,1608,Andaman & Nicobar
3,andaman-&-nicobar-islands,2018,1,pincode,744104,272,Andaman & Nicobar
4,andaman-&-nicobar-islands,2018,1,pincode,744105,1075,Andaman & Nicobar


Shape: (10000, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   state_raw         10000 non-null  object
 1   year              10000 non-null  int64 
 2   quarter           10000 non-null  int64 
 3   entity_type       10000 non-null  object
 4   entity_name       10000 non-null  object
 5   registered_users  10000 non-null  int64 
 6   parent_state      10000 non-null  object
dtypes: int64(3), object(4)
memory usage: 547.0+ KB
Duplicate rows: 0


In [10]:
# -----------------------------
# 0. CLEAR OLD VARIABLES
# -----------------------------
if "df_map_insurance" in globals():
    del df_map_insurance
if "map_insurance_records" in globals():
    del map_insurance_records

# -----------------------------
# 1. SET CORRECT BASE PATH
# -----------------------------
# 👉 CHANGE THIS TO YOUR REAL PATH (similar to your map_transaction path)
BASE_PATH_MAP_INS = r"E:/Guvi/Proj/Assignment 1/pp/pulse/data/map/insurance/hover/country/india/state"

# Quick debug: does this path exist?
print("Path exists? ->", os.path.exists(BASE_PATH_MAP_INS))
if os.path.exists(BASE_PATH_MAP_INS):
    print("Sample entries in base path:", os.listdir(BASE_PATH_MAP_INS)[:5])

# -----------------------------
# 2. COLLECT RECORDS
# -----------------------------
map_insurance_records = []

for state_folder in os.listdir(BASE_PATH_MAP_INS):
    state_path = os.path.join(BASE_PATH_MAP_INS, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        try:
            year = int(year_folder)
        except ValueError:
            continue

        for filename in os.listdir(year_path):
            if not filename.endswith(".json"):
                continue

            quarter_str = os.path.splitext(filename)[0]
            try:
                quarter = int(quarter_str)
            except ValueError:
                continue

            file_path = os.path.join(year_path, filename)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # For map/insurance, structure should be similar to map/transaction:
            # data["data"]["hoverDataList"]
            hover_list = data.get("data", {}).get("hoverDataList", []) or []
            if not hover_list:
                # Uncomment this if you want to debug empty files
                # print("No hoverDataList in:", file_path)
                continue

            for entry in hover_list:
                district_name = entry.get("name")
                metrics = entry.get("metric", [])
                if not metrics:
                    continue

                ins_count = metrics[0].get("count")
                ins_amount = metrics[0].get("amount")

                map_insurance_records.append({
                    "state_raw": state_folder,
                    "year": year,
                    "quarter": quarter,
                    "district": district_name,
                    "ins_count": ins_count,
                    "ins_amount": ins_amount
                })

# -----------------------------
# 3. BUILD DATAFRAME
# -----------------------------
df_map_insurance = pd.DataFrame(map_insurance_records)

print("Rows collected:", len(df_map_insurance))
display(df_map_insurance.head())
df_map_insurance.info()

# -----------------------------
# 4. CLEAN STATE NAMES
# -----------------------------
if not df_map_insurance.empty:
    df_map_insurance["state"] = (
        df_map_insurance["state_raw"]
        .str.replace("andaman-&-nicobar-islands", "Andaman & Nicobar", regex=False)
        .str.replace("-", " ", regex=False)
        .str.title()
    )

    df_map_insurance["state"] = df_map_insurance["state"].str.replace(
        "Dadra & Nagar Haveli & Daman & Diu",
        "Dadra And Nagar Haveli And Daman And Diu",
        regex=False
    )

    df_map_insurance["year"] = df_map_insurance["year"].astype(int)
    df_map_insurance["quarter"] = df_map_insurance["quarter"].astype(int)
    df_map_insurance["ins_count"] = df_map_insurance["ins_count"].astype("int64")
    df_map_insurance["ins_amount"] = df_map_insurance["ins_amount"].astype("float64")

    df_map_insurance = df_map_insurance.drop_duplicates().sort_values(
        by=["state", "year", "quarter", "district"]
    ).reset_index(drop=True)

    print("Final shape:", df_map_insurance.shape)
    display(df_map_insurance.head())
else:
    print("df_map_insurance is empty – check BASE_PATH_MAP_INS or JSON structure.")


Path exists? -> True
Sample entries in base path: ['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh', 'assam', 'bihar']
Rows collected: 13876


,state_raw,year,quarter,district,ins_count,ins_amount
0,andaman-&-nicobar-islands,2020,2,south andaman district,3,795.0
1,andaman-&-nicobar-islands,2020,2,nicobars district,3,565.0
2,andaman-&-nicobar-islands,2020,3,north and middle andaman district,1,281.0
3,andaman-&-nicobar-islands,2020,3,south andaman district,35,13651.0
4,andaman-&-nicobar-islands,2020,3,nicobars district,5,1448.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13876 entries, 0 to 13875
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_raw   13876 non-null  object 
 1   year        13876 non-null  int64  
 2   quarter     13876 non-null  int64  
 3   district    13876 non-null  object 
 4   ins_count   13876 non-null  int64  
 5   ins_amount  13876 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 650.6+ KB
Final shape: (13876, 7)


,state_raw,year,quarter,district,ins_count,ins_amount,state
0,andaman-&-nicobar-islands,2020,2,nicobars district,3,565.0,Andaman & Nicobar
1,andaman-&-nicobar-islands,2020,2,south andaman district,3,795.0,Andaman & Nicobar
2,andaman-&-nicobar-islands,2020,3,nicobars district,5,1448.0,Andaman & Nicobar
3,andaman-&-nicobar-islands,2020,3,north and middle andaman district,1,281.0,Andaman & Nicobar
4,andaman-&-nicobar-islands,2020,3,south andaman district,35,13651.0,Andaman & Nicobar


In [12]:
# MySQL connection details – CHANGE password if needed
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "root"  # <-- change this
DB_NAME = "phonepe_data"
DB_PORT = 3306  # default MySQL port

connection = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    port=DB_PORT,
    autocommit=False
)

cursor = connection.cursor()
print("Connected to MySQL:", DB_NAME)

Connected to MySQL: phonepe_data


In [4]:
create_agg_txn = """
CREATE TABLE IF NOT EXISTS aggregated_transaction (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    year INT,
    quarter TINYINT,
    txn_type VARCHAR(100),
    txn_count BIGINT,
    txn_amount DOUBLE
);
"""
cursor.execute(create_agg_txn)
connection.commit()

In [5]:
create_agg_user = """
CREATE TABLE IF NOT EXISTS aggregated_user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    year INT,
    quarter TINYINT,
    device_brand VARCHAR(100),
    device_user_count BIGINT,
    device_percentage DOUBLE,
    total_registered_users BIGINT NULL,
    total_app_opens BIGINT NULL
);
"""
cursor.execute(create_agg_user)
connection.commit()

In [6]:
create_agg_ins = """
CREATE TABLE IF NOT EXISTS aggregated_insurance (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    year INT,
    quarter TINYINT,
    ins_type VARCHAR(100),
    ins_count BIGINT,
    ins_amount DOUBLE
);
"""
cursor.execute(create_agg_ins)
connection.commit()

In [7]:
create_map_txn = """
CREATE TABLE IF NOT EXISTS map_transaction (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    year INT,
    quarter TINYINT,
    district VARCHAR(150),
    district_txn_count BIGINT,
    district_txn_amount DOUBLE
);
"""
cursor.execute(create_map_txn)
connection.commit()

In [8]:
create_map_user = """
CREATE TABLE IF NOT EXISTS map_user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    year INT,
    quarter TINYINT,
    district VARCHAR(150),
    registered_users BIGINT,
    app_opens BIGINT
);
"""
cursor.execute(create_map_user)
connection.commit()

In [9]:
create_top_txn = """
CREATE TABLE IF NOT EXISTS top_transaction (
    id INT AUTO_INCREMENT PRIMARY KEY,
    parent_state VARCHAR(100),
    year INT,
    quarter TINYINT,
    entity_type VARCHAR(20),
    entity_name VARCHAR(150),
    txn_count BIGINT,
    txn_amount DOUBLE
);
"""
cursor.execute(create_top_txn)
connection.commit()

In [10]:
create_top_user = """
CREATE TABLE IF NOT EXISTS top_user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    parent_state VARCHAR(100),
    year INT,
    quarter TINYINT,
    entity_type VARCHAR(20),
    entity_name VARCHAR(150),
    registered_users BIGINT
);
"""
cursor.execute(create_top_user)
connection.commit()

In [13]:
create_map_insurance_table = """
CREATE TABLE IF NOT EXISTS map_insurance (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    year INT,
    quarter TINYINT,
    district VARCHAR(150),
    ins_count BIGINT,
    ins_amount DOUBLE
);
"""

cursor.execute(create_map_insurance_table)
connection.commit()

In [10]:
insert_agg_txn = """
INSERT INTO aggregated_transaction 
(state, year, quarter, txn_type, txn_count, txn_amount)
VALUES (%s, %s, %s, %s, %s, %s);
"""

data_agg_txn = df_agg_transaction[["state", "year", "quarter", "txn_type", "txn_count", "txn_amount"]].values.tolist()

cursor.executemany(insert_agg_txn, data_agg_txn)
connection.commit()
print("Inserted rows into aggregated_transaction:", cursor.rowcount)

Inserted rows into aggregated_transaction: 5034


In [11]:
insert_agg_user = """
INSERT INTO aggregated_user
(state, year, quarter, device_brand, device_user_count, device_percentage, total_registered_users, total_app_opens)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""

data_agg_user = df_agg_user[[
    "state", "year", "quarter", "device_brand",
    "device_user_count", "device_percentage",
    "total_registered_users", "total_app_opens"
]].values.tolist()

cursor.executemany(insert_agg_user, data_agg_user)
connection.commit()
print("Inserted rows into aggregated_user:", cursor.rowcount)

Inserted rows into aggregated_user: 6732


In [12]:
insert_agg_ins = """
INSERT INTO aggregated_insurance
(state, year, quarter, ins_type, ins_count, ins_amount)
VALUES (%s, %s, %s, %s, %s, %s);
"""

data_agg_ins = df_agg_insurance[[
    "state", "year", "quarter", "ins_type", "ins_count", "ins_amount"
]].values.tolist()

cursor.executemany(insert_agg_ins, data_agg_ins)
connection.commit()
print("Inserted rows into aggregated_insurance:", cursor.rowcount)

Inserted rows into aggregated_insurance: 682


In [13]:
insert_map_txn = """
INSERT INTO map_transaction
(state, year, quarter, district, district_txn_count, district_txn_amount)
VALUES (%s, %s, %s, %s, %s, %s);
"""

data_map_txn = df_map_transaction[[
    "state", "year", "quarter",
    "district", "district_txn_count", "district_txn_amount"
]].values.tolist()

cursor.executemany(insert_map_txn, data_map_txn)
connection.commit()
print("Inserted rows into map_transaction:", cursor.rowcount)

Inserted rows into map_transaction: 20604


In [14]:
insert_map_user = """
INSERT INTO map_user
(state, year, quarter, district, registered_users, app_opens)
VALUES (%s, %s, %s, %s, %s, %s);
"""

data_map_user = df_map_user[[
    "state", "year", "quarter",
    "district", "registered_users", "app_opens"
]].values.tolist()

cursor.executemany(insert_map_user, data_map_user)
connection.commit()
print("Inserted rows into map_user:", cursor.rowcount)

Inserted rows into map_user: 20608


In [15]:
insert_top_txn = """
INSERT INTO top_transaction
(parent_state, year, quarter, entity_type, entity_name, txn_count, txn_amount)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

data_top_txn = df_top_transaction[[
    "parent_state", "year", "quarter",
    "entity_type", "entity_name", "txn_count", "txn_amount"
]].values.tolist()

cursor.executemany(insert_top_txn, data_top_txn)
connection.commit()
print("Inserted rows into top_transaction:", cursor.rowcount)

Inserted rows into top_transaction: 18295


In [18]:
insert_top_user = """
INSERT INTO top_user
(parent_state, year, quarter, entity_type, entity_name, registered_users)
VALUES (%s, %s, %s, %s, %s, %s);
"""

data_top_user = df_top_user[[
    "parent_state", "year", "quarter",
    "entity_type", "entity_name", "registered_users"
]].values.tolist()

cursor.executemany(insert_top_user, data_top_user)
connection.commit()
print("Inserted rows into top_user:", cursor.rowcount)

Inserted rows into top_user: 10000


In [14]:
insert_map_insurance = """
INSERT INTO map_insurance
(state, year, quarter, district, ins_count, ins_amount)
VALUES (%s, %s, %s, %s, %s, %s);
"""

data_map_insurance = df_map_insurance[[
    "state", "year", "quarter", "district", "ins_count", "ins_amount"
]].values.tolist()

cursor.executemany(insert_map_insurance, data_map_insurance)
connection.commit()
print("Inserted rows into map_insurance:", cursor.rowcount)

Inserted rows into map_insurance: 13876


In [ ]:
# Scenario1 - Decoding Transaction Dynamics

# 1. SQL query for yearly transaction trend
query_yearly_txn = """
SELECT 
    year,
    SUM(txn_count)  AS total_txn_count,
    SUM(txn_amount) AS total_txn_amount
FROM aggregated_transaction
GROUP BY year
ORDER BY year;
"""

# 2. Read into DataFrame
df_yearly_txn = pd.read_sql(query_yearly_txn, connection)
display(df_yearly_txn)

# 3. Line chart for total transaction amount by year
fig_amount = px.line(
    df_yearly_txn,
    x="year",
    y="total_txn_amount",
    markers=True,
    title="Total PhonePe Transaction Amount by Year (All India)"
)
fig_amount.update_layout(xaxis_title="Year", yaxis_title="Total Transaction Amount (₹)")
fig_amount.show()

# 4. Optional: Line chart for total transaction count by year
fig_count = px.line(
    df_yearly_txn,
    x="year",
    y="total_txn_count",
    markers=True,
    title="Total PhonePe Transaction Count by Year (All India)"
)
fig_count.update_layout(xaxis_title="Year", yaxis_title="Total Transaction Count")
fig_count.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\3956649023.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,total_txn_count,total_txn_amount
0,2018,1.080202e+09,1.623045e+12
1,2019,4.079827e+09,6.276688e+12
2,2020,7.973975e+09,1.464116e+13
3,2021,1.928843e+10,3.459870e+13
4,2022,3.930129e+10,6.426633e+13
5,2023,6.425705e+10,9.449181e+13
6,2024,9.930343e+10,1.296246e+14


In [22]:
# 1) SQL: Year + Transaction Type + Totals
query_txn_type_trend = """
SELECT 
    year,
    txn_type,
    SUM(txn_count)  AS total_txn_count,
    SUM(txn_amount) AS total_txn_amount
FROM aggregated_transaction
GROUP BY year, txn_type
ORDER BY year, total_txn_amount DESC;
"""

# 2) Load into DataFrame
df_txn_type_trend = pd.read_sql(query_txn_type_trend, connection)
display(df_txn_type_trend.head())
print("Shape:", df_txn_type_trend.shape)

# 3) Multi-line chart: Transaction Amount trend by payment type
fig_txn_type_line = px.line(
    df_txn_type_trend,
    x="year",
    y="total_txn_amount",
    color="txn_type",
    markers=True,
    title="Transaction Amount Trend by Payment Type (All India)",
    labels={
        "year": "Year",
        "total_txn_amount": "Total Transaction Amount",
        "txn_type": "Payment Type"
    }
)
fig_txn_type_line.update_layout(xaxis=dict(dtick=1))
fig_txn_type_line.show()

# 4) Stacked bar chart: Contribution of each payment type per year
fig_txn_type_bar = px.bar(
    df_txn_type_trend,
    x="year",
    y="total_txn_amount",
    color="txn_type",
    title="Yearly Contribution of Payment Types (Transaction Amount)",
    labels={
        "year": "Year",
        "total_txn_amount": "Total Transaction Amount",
        "txn_type": "Payment Type"
    },
    barmode="stack"
)
fig_txn_type_bar.update_layout(xaxis=dict(dtick=1))
fig_txn_type_bar.show()

# 5) Optional: Count-based line chart (if you want to see volume trend)
fig_txn_count_line = px.line(
    df_txn_type_trend,
    x="year",
    y="total_txn_count",
    color="txn_type",
    markers=True,
    title="Transaction Count Trend by Payment Type (All India)",
    labels={
        "year": "Year",
        "total_txn_count": "Total Transaction Count",
        "txn_type": "Payment Type"
    }
)
fig_txn_count_line.update_layout(xaxis=dict(dtick=1))
fig_txn_count_line.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\1632936526.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,txn_type,total_txn_count,total_txn_amount
0,2018,Peer-to-peer payments,523796208.0,1.432964e+12
1,2018,Recharge & bill payments,398789466.0,1.097659e+11
2,2018,Merchant payments,112656898.0,5.663979e+10
3,2018,Others,24575693.0,1.992282e+10
4,2018,Financial Services,20384145.0,3.752877e+09


Shape: (35, 4)


In [23]:
# 1) SQL: State + Year + totals
query_state_year_txn = """
SELECT 
    state,
    year,
    SUM(txn_count)  AS total_txn_count,
    SUM(txn_amount) AS total_txn_amount
FROM aggregated_transaction
GROUP BY state, year
ORDER BY year, total_txn_amount DESC;
"""

# 2) Load into DataFrame
df_state_year_txn = pd.read_sql(query_state_year_txn, connection)
display(df_state_year_txn.head())
print("Shape:", df_state_year_txn.shape)

# ---------------------------------
# 3) Bar chart: Top N states by transaction amount for a selected year
# ---------------------------------

# 🔹 You can change this year value to see different years
selected_year = 2022   # <- EDIT this as you like (2018–2024)

df_year = df_state_year_txn[df_state_year_txn["year"] == selected_year].copy()

# If there are no rows for that year, handle gracefully
if df_year.empty:
    print(f"No data available for year {selected_year}")
else:
    # Take top 10 states by transaction amount
    df_top_states = df_year.nlargest(10, "total_txn_amount")

    fig_top_states = px.bar(
        df_top_states,
        x="state",
        y="total_txn_amount",
        title=f"Top 10 States by Transaction Amount in {selected_year}",
        labels={
            "state": "State",
            "total_txn_amount": "Total Transaction Amount"
        }
    )
    fig_top_states.update_layout(xaxis_tickangle=-45)
    fig_top_states.show()

# ---------------------------------
# 4) Line chart: Transaction Amount Trend over Years for Top States
# ---------------------------------

# 🔹 Find top N states overall (by total amount across all years)
N_TOP_STATES = 5

df_total_by_state = (
    df_state_year_txn.groupby("state", as_index=False)["total_txn_amount"]
    .sum()
    .sort_values("total_txn_amount", ascending=False)
)

top_states_list = df_total_by_state["state"].head(N_TOP_STATES).tolist()
print("Top states overall:", top_states_list)

# Filter data for only these states
df_top_states_trend = df_state_year_txn[df_state_year_txn["state"].isin(top_states_list)].copy()

fig_trend_states = px.line(
    df_top_states_trend,
    x="year",
    y="total_txn_amount",
    color="state",
    markers=True,
    title=f"Transaction Amount Trend Over Years – Top {N_TOP_STATES} States",
    labels={
        "year": "Year",
        "total_txn_amount": "Total Transaction Amount",
        "state": "State"
    }
)
fig_trend_states.update_layout(xaxis=dict(dtick=1))
fig_trend_states.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\2033783916.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,year,total_txn_count,total_txn_amount
0,Maharashtra,2018,116301535.0,1.898484e+11
1,Karnataka,2018,99925678.0,1.739670e+11
2,Uttar Pradesh,2018,92864871.0,1.317690e+11
3,Andhra Pradesh,2018,77779112.0,1.220720e+11
4,Telangana,2018,67541293.0,1.178646e+11


Shape: (252, 4)


Top states overall: ['Telangana', 'Karnataka', 'Maharashtra', 'Andhra Pradesh', 'Uttar Pradesh']


In [25]:
# 1) SQL: Year + Quarter + Totals
query_quarter_trend = """
SELECT 
    year,
    quarter,
    SUM(txn_count)  AS total_txn_count,
    SUM(txn_amount) AS total_txn_amount
FROM aggregated_transaction
GROUP BY year, quarter
ORDER BY year, quarter;
"""

# 2) Load into DataFrame
df_quarter_trend = pd.read_sql(query_quarter_trend, connection)
display(df_quarter_trend.head())
print("Shape:", df_quarter_trend.shape)


# -------------------------------------------
# 3) Line Chart – Quarter Trends Across Years
# -------------------------------------------
fig_quarter_line = px.line(
    df_quarter_trend,
    x="quarter",
    y="total_txn_amount",
    color="year",
    markers=True,
    title="Quarterly Transaction Amount Trend (Year-over-Year)",
    labels={
        "quarter": "Quarter",
        "total_txn_amount": "Total Transaction Amount",
        "year": "Year"
    }
)

fig_quarter_line.update_layout(
    xaxis=dict(dtick=1),
    legend_title="Year",
    width=900,
    height=500
)

fig_quarter_line.show()


# -------------------------------------------
# 4) Heatmap – Seasonal Patterns Across Years
# -------------------------------------------
df_heatmap = df_quarter_trend.pivot(
    index="year", 
    columns="quarter", 
    values="total_txn_amount"
)

fig_heatmap = px.imshow(
    df_heatmap,
    labels=dict(x="Quarter", y="Year", color="Transaction Amount"),
    title="Quarterly Transaction Amount Heatmap (Seasonality Analysis)",
    color_continuous_scale="Viridis"
)

fig_heatmap.update_layout(width=700, height=500)
fig_heatmap.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\2674704842.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,quarter,total_txn_count,total_txn_amount
0,2018,1,134425599.0,1.718334e+11
1,2018,2,187365440.0,3.043742e+11
2,2018,3,341299764.0,4.751015e+11
3,2018,4,417111607.0,6.717362e+11
4,2019,1,708992981.0,9.900214e+11


Shape: (28, 4)


In [28]:
#Scenario2 - Device Dominance & User Engagement Analysis

query_brand_trend = """
SELECT
    year,
    device_brand AS brand,
    SUM(device_user_count) AS total_registered_users
FROM aggregated_user
GROUP BY year, device_brand
ORDER BY year, total_registered_users DESC;
"""

df_brand_trend = pd.read_sql(query_brand_trend, connection)
display(df_brand_trend.head())
print("Shape:", df_brand_trend.shape)

# -------------------------------------
# MULTI-LINE CHART: Brand Trend YOY
# -------------------------------------

fig_brand_line = px.line(
    df_brand_trend,
    x="year",
    y="total_registered_users",
    color="brand",
    markers=True,
    title="Mobile Brand Usage Trend Across Years",
    labels={
        "year": "Year",
        "brand": "Device Brand",
        "total_registered_users": "Registered Users"
    }
)

fig_brand_line.update_layout(
    xaxis=dict(dtick=1),
    width=900,
    height=500
)

fig_brand_line.show()

# -------------------------------------
# BAR CHART: Top 10 Brands by Total Users (Overall)
# -------------------------------------

df_brand_overall = (
    df_brand_trend
    .groupby("brand")["total_registered_users"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

fig_brand_bar = px.bar(
    df_brand_overall.head(10),
    x="brand",
    y="total_registered_users",
    title="Top 10 Mobile Brands Across India (Registered Users)",
    labels={"total_registered_users": "Registered Users", "brand": "Device Brand"}
)

fig_brand_bar.update_layout(xaxis_tickangle=-45)
fig_brand_bar.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\3619336963.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,brand,total_registered_users
0,2018,Xiaomi,75911526.0
1,2018,Samsung,59520848.0
2,2018,Vivo,40344921.0
3,2018,Others,31483633.0
4,2018,Oppo,29583532.0


Shape: (87, 3)


In [29]:
# -------------------------------------
# SCENARIO 2 – PART 2:
# State-wise Device Brand Usage (Registered Users)
# -------------------------------------

# 👉 You can change these as you like
selected_year = 2022        # any year present in your data: 2018–2024
selected_state = "Tamil Nadu"  # must match state name in table


# 1) Get state-year-brand aggregated data from MySQL
query_state_brand_year = """
SELECT
    state,
    year,
    device_brand AS brand,
    SUM(device_user_count) AS total_users
FROM aggregated_user
GROUP BY state, year, device_brand;
"""

df_state_brand = pd.read_sql(query_state_brand_year, connection)
print("Full shape:", df_state_brand.shape)
display(df_state_brand.head())


# 2) Filter for selected year
df_year = df_state_brand[df_state_brand["year"] == selected_year].copy()

if df_year.empty:
    print(f"No data found for year {selected_year}")
else:
    print(f"Rows for year {selected_year}:", df_year.shape[0])

    # -------------------------------------
    # 2A) Bar chart: Brand usage within a selected state
    # -------------------------------------
    df_state = df_year[df_year["state"] == selected_state].copy()

    if df_state.empty:
        print(f"No data found for state '{selected_state}' in year {selected_year}")
    else:
        df_state_sorted = df_state.sort_values("total_users", ascending=False)

        fig_state_brand = px.bar(
            df_state_sorted,
            x="brand",
            y="total_users",
            title=f"Device Brand Usage in {selected_state} – {selected_year}",
            labels={
                "brand": "Device Brand",
                "total_users": "Registered Users"
            }
        )
        fig_state_brand.update_layout(xaxis_tickangle=-45, width=800, height=450)
        fig_state_brand.show()

    # -------------------------------------
    # 2B) Bar chart: Top brands across ALL states for that year
    # -------------------------------------
    df_brand_year = (
        df_year.groupby("brand", as_index=False)["total_users"]
        .sum()
        .sort_values("total_users", ascending=False)
    )

    fig_brand_year = px.bar(
        df_brand_year.head(10),
        x="brand",
        y="total_users",
        title=f"Top 10 Device Brands Across India – {selected_year}",
        labels={
            "brand": "Device Brand",
            "total_users": "Registered Users"
        }
    )
    fig_brand_year.update_layout(xaxis_tickangle=-45, width=800, height=450)
    fig_brand_year.show()

    # -------------------------------------
    # 2C) Heatmap: States vs Brands (who prefers what?)
    # -------------------------------------
    df_heatmap = df_year.pivot(
        index="state",
        columns="brand",
        values="total_users"
    )

    fig_heatmap = px.imshow(
        df_heatmap,
        labels={
            "x": "Device Brand",
            "y": "State",
            "color": "Registered Users"
        },
        title=f"State vs Device Brand – Registered Users Heatmap ({selected_year})"
    )

    fig_heatmap.update_layout(width=900, height=700)
    fig_heatmap.show()


Full shape: (2049, 4)


C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\3901686770.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,year,brand,total_users
0,Andaman & Nicobar,2018,Apple,1363.0
1,Andaman & Nicobar,2018,Huawei,1100.0
2,Andaman & Nicobar,2018,Lenovo,1122.0
3,Andaman & Nicobar,2018,Motorola,1409.0
4,Andaman & Nicobar,2018,OnePlus,2136.0


Rows for year 2022: 396


In [30]:
# -------------------------------------
# SCENARIO 2 – PART 3:
# User Engagement = AppOpens vs Registered Users
# Using aggregated_user table
# -------------------------------------

# 1) SQL: Aggregate to state + year level
query_engagement = """
SELECT
    state,
    year,
    SUM(total_registered_users) AS total_registered_users,
    SUM(total_app_opens)       AS total_app_opens
FROM aggregated_user
WHERE total_registered_users IS NOT NULL
  AND total_app_opens IS NOT NULL
GROUP BY state, year
HAVING SUM(total_registered_users) > 0;
"""

df_engagement = pd.read_sql(query_engagement, connection)
display(df_engagement.head())
print("Shape:", df_engagement.shape)

# 2) Compute engagement ratio
df_engagement["engagement_ratio"] = (
    df_engagement["total_app_opens"] / df_engagement["total_registered_users"]
)

display(df_engagement.head())


# -------------------------------------
# 3) Select a year to analyze
# -------------------------------------
selected_year = 2022   # 🔁 change as you like: 2019, 2020, 2021, 2023, etc.

df_eng_year = df_engagement[df_engagement["year"] == selected_year].copy()

if df_eng_year.empty:
    print(f"No engagement data available for year {selected_year}")
else:
    print(f"States in {selected_year}:", df_eng_year.shape[0])

    # -------------------------------------
    # 3A) TOP 10 STATES BY ENGAGEMENT RATIO
    # -------------------------------------
    df_top_eng = df_eng_year.sort_values(
        "engagement_ratio", ascending=False
    ).head(10)

    fig_top_eng = px.bar(
        df_top_eng,
        x="state",
        y="engagement_ratio",
        title=f"Top 10 States by Engagement Ratio ({selected_year})",
        labels={
            "state": "State",
            "engagement_ratio": "Engagement (AppOpens / Registered User)"
        }
    )
    fig_top_eng.update_layout(xaxis_tickangle=-45, width=900, height=450)
    fig_top_eng.show()

    # -------------------------------------
    # 3B) BOTTOM 10 STATES BY ENGAGEMENT RATIO
    # -------------------------------------
    df_bottom_eng = df_eng_year.sort_values(
        "engagement_ratio", ascending=True
    ).head(10)

    fig_bottom_eng = px.bar(
        df_bottom_eng,
        x="state",
        y="engagement_ratio",
        title=f"Bottom 10 States by Engagement Ratio ({selected_year})",
        labels={
            "state": "State",
            "engagement_ratio": "Engagement (AppOpens / Registered User)"
        }
    )
    fig_bottom_eng.update_layout(xaxis_tickangle=-45, width=900, height=450)
    fig_bottom_eng.show()

    # -------------------------------------
    # 3C) Scatter: Registered Users vs AppOpens (Color = Engagement)
    # -------------------------------------
    fig_scatter = px.scatter(
        df_eng_year,
        x="total_registered_users",
        y="total_app_opens",
        color="engagement_ratio",
        hover_name="state",
        title=f"State Engagement Profile – Registered Users vs AppOpens ({selected_year})",
        labels={
            "total_registered_users": "Total Registered Users",
            "total_app_opens": "Total App Opens",
            "engagement_ratio": "Engagement Ratio"
        }
    )

    fig_scatter.update_layout(width=900, height=500)
    fig_scatter.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\1250179485.py:21: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,year,total_registered_users,total_app_opens
0,Andaman & Nicobar,2018,478676.0,0.0
1,Andaman & Nicobar,2019,992398.0,4408349.0
2,Andaman & Nicobar,2020,1702899.0,20915741.0
3,Andaman & Nicobar,2021,2844765.0,93358034.0
4,Andaman & Nicobar,2022,839168.0,42321477.0


Shape: (180, 4)


,state,year,total_registered_users,total_app_opens,engagement_ratio
0,Andaman & Nicobar,2018,478676.0,0.0,0.000000
1,Andaman & Nicobar,2019,992398.0,4408349.0,4.442118
2,Andaman & Nicobar,2020,1702899.0,20915741.0,12.282432
3,Andaman & Nicobar,2021,2844765.0,93358034.0,32.817485
4,Andaman & Nicobar,2022,839168.0,42321477.0,50.432663


States in 2022: 36


In [32]:
# -------------------------------------
# SCENARIO 3 – PART 1:
# National Insurance Growth (Year-wise)
# -------------------------------------

query_insurance_year = """
SELECT
    year,
    SUM(ins_count)  AS total_insurance_policies,
    SUM(ins_amount) AS total_insurance_value
FROM aggregated_insurance
GROUP BY year
ORDER BY year;
"""

df_ins_year = pd.read_sql(query_insurance_year, connection)
display(df_ins_year.head())
print("Shape:", df_ins_year.shape)


# -------------------------------------
# 1) Line Chart — Insurance Amount Growth
# -------------------------------------
fig_ins_amount = px.line(
    df_ins_year,
    x="year",
    y="total_insurance_value",
    markers=True,
    title="Yearly Insurance Transaction Value Growth (India)",
    labels={
        "year": "Year",
        "total_insurance_value": "Insurance Value (₹)"
    }
)
fig_ins_amount.update_layout(xaxis=dict(dtick=1), width=900, height=450)
fig_ins_amount.show()

# -------------------------------------
# 2) Line Chart — Number of Insurance Policies
# -------------------------------------
fig_ins_count = px.line(
    df_ins_year,
    x="year",
    y="total_insurance_policies",
    markers=True,
    title="Yearly Insurance Policy Count Growth (India)",
    labels={
        "year": "Year",
        "total_insurance_policies": "Number of Policies"
    }
)
fig_ins_count.update_layout(xaxis=dict(dtick=1), width=900, height=450)
fig_ins_count.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\1978591266.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,year,total_insurance_policies,total_insurance_value
0,2020,788258.0,2.942072e+08
1,2021,1583282.0,1.499170e+09
2,2022,3090116.0,4.238843e+09
3,2023,3986187.0,6.056606e+09
4,2024,5067844.0,7.926752e+09


Shape: (5, 3)


In [36]:
# -------------------------------------
# SCENARIO 3 – PART 2:
# State-wise Insurance Adoption
# -------------------------------------

query_ins_state = """
SELECT
    state,
    year,
    SUM(ins_count) AS total_insurance_policies,
    SUM(ins_amount) AS total_insurance_value
FROM aggregated_insurance
GROUP BY state, year
ORDER BY total_insurance_value DESC;
"""

df_ins_state = pd.read_sql(query_ins_state, connection)
display(df_ins_state.head())


# -------------------------------------
# Choose a year for comparison
# -------------------------------------
selected_year = 2022   # change as needed

df_ins_state_year = df_ins_state[df_ins_state["year"] == selected_year].copy()

# Top 10 states
df_top_ins_states = df_ins_state_year.nlargest(10, "total_insurance_value")

fig_top_states = px.bar(
    df_top_ins_states,
    x="state",
    y="total_insurance_value",
    title=f"Top 10 States by Insurance Value in {selected_year}",
    labels={"state": "State", "total_insurance_value": "Insurance Value (₹)"}
)
fig_top_states.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_top_states.show()


# Bottom 10 states
df_bottom_ins_states = df_ins_state_year.nsmallest(10, "total_insurance_value")

fig_bottom_states = px.bar(
    df_bottom_ins_states,
    x="state",
    y="total_insurance_value",
    title=f"Bottom 10 States by Insurance Value in {selected_year}",
    labels={"state": "State", "total_insurance_value": "Insurance Value (₹)"}
)
fig_bottom_states.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_bottom_states.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_15656\187430145.py:17: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,year,total_insurance_policies,total_insurance_value
0,Karnataka,2024,670478.0,1.097269e+09
1,Maharashtra,2024,555803.0,8.919331e+08
2,Karnataka,2023,540383.0,8.097523e+08
3,Uttar Pradesh,2024,405678.0,6.928204e+08
4,Maharashtra,2023,456768.0,6.927286e+08


In [15]:
# -------------------------------------
# SCENARIO 3 – PART 3:
# District-level Insurance Analysis
# Source: map_insurance (state, year, quarter, district, ins_count, ins_amount)
# -------------------------------------

# 1) Read aggregated district-level insurance data
query_ins_district = """
SELECT
    state,
    district,
    year,
    SUM(ins_count)  AS total_insurance_policies,
    SUM(ins_amount) AS total_insurance_value
FROM map_insurance
GROUP BY state, district, year;
"""

df_ins_dist = pd.read_sql(query_ins_district, connection)
display(df_ins_dist.head())
print("Shape:", df_ins_dist.shape)

# 2) Choose a year for analysis
selected_year = 2022   # 🔁 change as needed (e.g. 2020, 2021, 2023...)

df_dist_year = df_ins_dist[df_ins_dist["year"] == selected_year].copy()

if df_dist_year.empty:
    print(f"No district-level insurance data found for year {selected_year}")
else:
    print(f"District rows for {selected_year}:", df_dist_year.shape[0])

    # -------------------------------------
    # 3A) TOP 10 DISTRICTS by Insurance Value
    # -------------------------------------
    df_top_districts = df_dist_year.nlargest(10, "total_insurance_value")

    fig_top_districts = px.bar(
        df_top_districts,
        x="district",
        y="total_insurance_value",
        color="state",
        title=f"Top 10 Districts by Insurance Value ({selected_year})",
        labels={
            "district": "District",
            "total_insurance_value": "Insurance Value",
            "state": "State"
        }
    )
    fig_top_districts.update_layout(xaxis_tickangle=-45, width=900, height=450)
    fig_top_districts.show()

    # -------------------------------------
    # 3B) BOTTOM 10 DISTRICTS by Insurance Value
    # -------------------------------------
    df_bottom_districts = df_dist_year.nsmallest(10, "total_insurance_value")

    fig_bottom_districts = px.bar(
        df_bottom_districts,
        x="district",
        y="total_insurance_value",
        color="state",
        title=f"Bottom 10 Districts by Insurance Value ({selected_year})",
        labels={
            "district": "District",
            "total_insurance_value": "Insurance Value",
            "state": "State"
        }
    )
    fig_bottom_districts.update_layout(xaxis_tickangle=-45, width=900, height=450)
    fig_bottom_districts.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_3760\3935823572.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ins_dist = pd.read_sql(query_ins_district, connection)


,state,district,year,total_insurance_policies,total_insurance_value
0,Andaman & Nicobar,nicobars district,2020,10.0,4760.0
1,Andaman & Nicobar,south andaman district,2020,147.0,151940.0
2,Andaman & Nicobar,north and middle andaman district,2020,14.0,18015.0
3,Andaman & Nicobar,nicobars district,2021,76.0,132433.0
4,Andaman & Nicobar,north and middle andaman district,2021,190.0,342347.0


Shape: (3770, 5)
District rows for 2022: 731


In [16]:
# Scenario 4 - PART 1 — YEAR-ON-YEAR GROWTH (STATE GROWTH TREND)
# Load state-year transaction summary
query_state_year = """
SELECT 
    state,
    year,
    SUM(txn_count)  AS total_txn_count,
    SUM(txn_amount) AS total_txn_amount
FROM aggregated_transaction
GROUP BY state, year
ORDER BY state, year;
"""

df_state_year = pd.read_sql(query_state_year, connection)
display(df_state_year.head())

# Compute YoY growth %
df_state_year["txn_growth_pct"] = df_state_year.groupby("state")["total_txn_count"].pct_change() * 100
df_state_year["amount_growth_pct"] = df_state_year.groupby("state")["total_txn_amount"].pct_change() * 100

# Plot YoY Growth (Transaction Count %)
import plotly.express as px

latest_year = df_state_year["year"].max()

df_growth_latest = df_state_year[df_state_year["year"] == latest_year].sort_values(
    "txn_growth_pct", ascending=False
)

fig_growth = px.bar(
    df_growth_latest.head(10),
    x="state",
    y="txn_growth_pct",
    title=f"Top 10 Fastest Growing States (Transaction Count Growth %) – {latest_year}",
    labels={"txn_growth_pct": "Growth %", "state": "State"},
    color="txn_growth_pct",
)

fig_growth.update_layout(xaxis_tickangle=-45, width=950, height=450)
fig_growth.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_3760\3870506449.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,year,total_txn_count,total_txn_amount
0,Andaman & Nicobar,2018,58021.0,1.890761e+08
1,Andaman & Nicobar,2019,133104.0,4.734648e+08
2,Andaman & Nicobar,2020,446274.0,1.296423e+09
3,Andaman & Nicobar,2021,1672053.0,4.925488e+09
4,Andaman & Nicobar,2022,5437879.0,1.211026e+10


In [18]:
# Scenario 4 - PART 2 — MARKET SIZE ANALYSIS (WHICH STATES ARE BIG?)
query_state_market = """
SELECT
    state,
    SUM(txn_count)  AS total_txn_count,
    SUM(txn_amount) AS total_txn_amount
FROM aggregated_transaction
GROUP BY state
ORDER BY total_txn_amount DESC;
"""

df_market = pd.read_sql(query_state_market, connection)

fig_market = px.bar(
    df_market.head(10),
    x="state",
    y="total_txn_amount",
    title="Top 10 States by Total Transaction Value",
    color="total_txn_amount"
)

fig_market.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_market.show()

fig_low_market = px.bar(
    df_market.tail(10).sort_values("total_txn_amount"),
    x="state",
    y="total_txn_amount",
    title="Bottom 10 States by Transaction Value (Growth Opportunities)",
    color="total_txn_amount"
)

fig_low_market.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_low_market.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_3760\784308169.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [19]:
# Scenario 4 - PART 3 — GROWTH-POTENTIAL MATRIX (VERY IMPORTANT FOR MARKET EXPANSION)
# Normalize values for comparison
df_market["count_norm"] = (df_market["total_txn_count"] - df_market["total_txn_count"].min()) / \
                          (df_market["total_txn_count"].max() - df_market["total_txn_count"].min())

df_market["amount_norm"] = (df_market["total_txn_amount"] - df_market["total_txn_amount"].min()) / \
                           (df_market["total_txn_amount"].max() - df_market["total_txn_amount"].min())

# Define categories
df_market["category"] = df_market.apply(
    lambda row:
        "High Count – High Value" if row["count_norm"] >= 0.5 and row["amount_norm"] >= 0.5 else
        "High Count – Low Value"  if row["count_norm"] >= 0.5 and row["amount_norm"] < 0.5 else
        "Low Count – High Value"  if row["count_norm"] < 0.5 and row["amount_norm"] >= 0.5 else
        "Low Count – Low Value",
    axis=1
)

display(df_market)

# Plot the segmentation
fig_matrix = px.scatter(
    df_market,
    x="total_txn_count",
    y="total_txn_amount",
    color="category",
    hover_data=["state"],
    title="State Market Segmentation (Growth Potential Matrix)",
)

fig_matrix.update_layout(width=900, height=500)
fig_matrix.show()


,state,total_txn_count,total_txn_amount,count_norm,amount_norm,category
0,Telangana,2.617468e+10,4.165596e+13,0.818332,1.000000,High Count – High Value
1,Karnataka,3.097095e+10,4.067872e+13,0.968289,0.976539,High Count – High Value
2,Maharashtra,3.198521e+10,4.037420e+13,1.000000,0.969229,High Count – High Value
3,Andhra Pradesh,1.891870e+10,3.466908e+13,0.591471,0.832265,High Count – High Value
4,Uttar Pradesh,1.852360e+10,2.688521e+13,0.579119,0.645397,High Count – High Value
5,Rajasthan,1.710854e+10,2.634324e+13,0.534876,0.632386,High Count – High Value
6,Madhya Pradesh,1.407218e+10,1.912528e+13,0.439943,0.459104,Low Count – Low Value
7,Bihar,1.094103e+10,1.790135e+13,0.342047,0.429721,Low Count – Low Value
8,West Bengal,9.191500e+09,1.558416e+13,0.287347,0.374092,Low Count – Low Value
9,Odisha,8.918527e+09,1.226398e+13,0.278813,0.294384,Low Count – Low Value


In [20]:
# Scenario 5 - PART 1: State-Level User Engagement
query_user_state = """
SELECT
    state,
    SUM(registered_users) AS total_registered_users,
    SUM(app_opens) AS total_app_opens
FROM map_user
GROUP BY state
ORDER BY total_registered_users DESC;
"""

df_user_state = pd.read_sql(query_user_state, connection)
display(df_user_state.head())

import plotly.express as px

# Top 10 states by registered users
fig_user_state = px.bar(
    df_user_state.head(10),
    x="state",
    y="total_registered_users",
    color="total_registered_users",
    title="Top 10 States by Registered Users"
)
fig_user_state.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_user_state.show()

# Top 10 states by App Opens (Engagement)
df_user_state_sorted = df_user_state.sort_values("total_app_opens", ascending=False)

fig_app_state = px.bar(
    df_user_state_sorted.head(10),
    x="state",
    y="total_app_opens",
    color="total_app_opens",
    title="Top 10 States by App Opens (User Engagement)"
)
fig_app_state.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_app_state.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_3760\3488466216.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,total_registered_users,total_app_opens
0,Maharashtra,1.140138e+09,4.961642e+10
1,Uttar Pradesh,9.423341e+08,3.323683e+10
2,Karnataka,7.336742e+08,3.834488e+10
3,Andhra Pradesh,5.567471e+08,2.472802e+10
4,Rajasthan,5.557745e+08,4.850763e+10


In [21]:
# SCENARIO 5 — PART 2: District-Level Engagement Analysis
query_user_district = """
SELECT
    state,
    district,
    SUM(registered_users) AS total_registered_users,
    SUM(app_opens) AS total_app_opens
FROM map_user
GROUP BY state, district
ORDER BY total_registered_users DESC;
"""

df_user_dist = pd.read_sql(query_user_district, connection)

# Top districts by registered users
fig_user_dist = px.bar(
    df_user_dist.head(10),
    x="district",
    y="total_registered_users",
    color="state",
    title="Top 10 Districts by Registered Users"
)
fig_user_dist.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_user_dist.show()

# Top districts by app opens
df_user_dist_sorted = df_user_dist.sort_values("total_app_opens", ascending=False)

fig_app_dist = px.bar(
    df_user_dist_sorted.head(10),
    x="district",
    y="total_app_opens",
    color="state",
    title="Top 10 Districts by App Opens (Highest Engagement)"
)
fig_app_dist.update_layout(xaxis_tickangle=-45, width=900, height=450)
fig_app_dist.show()


C:\Users\rspmp\AppData\Local\Temp\ipykernel_3760\3769034755.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [24]:
# SCENARIO 5 — PART 3: Top Registration Hotspots (top_user)

query_top_user = """
SELECT
    parent_state AS state,
    entity_name  AS pincode,
    SUM(registered_users) AS total_registrations
FROM top_user
WHERE entity_type = 'pincode'
GROUP BY parent_state, entity_name
ORDER BY total_registrations DESC;
"""

df_top_reg = pd.read_sql(query_top_user, connection)
display(df_top_reg.head())
print("Shape:", df_top_reg.shape)

# Top 10 pincodes by registrations
fig_top_reg = px.bar(
    df_top_reg.head(10),
    x="pincode",
    y="total_registrations",
    color="state",
    title="Top 10 Pincodes by User Registrations",
    labels={
        "pincode": "Pincode",
        "total_registrations": "Total Registrations",
        "state": "State"
    }
)

fig_top_reg.update_layout(width=900, height=450)
fig_top_reg.show()

C:\Users\rspmp\AppData\Local\Temp\ipykernel_3760\2616226815.py:14: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,state,pincode,total_registrations
0,Uttar Pradesh,201301,16876987.0
1,Delhi,110059,14167569.0
2,Karnataka,560068,13002727.0
3,Maharashtra,421302,12366260.0
4,Telangana,500072,11771379.0


Shape: (515, 3)
